<a href="https://colab.research.google.com/github/AnudeepSatyaSai/AudioNest/blob/master/Open_Sora_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ninja colossalai mmengine
!pip install https://download.pytorch.org/whl/cu121/torch-2.2.1%2Bcu121-cp310-cp310-linux_x86_64.whl
!pip install https://download.pytorch.org/whl/cu121/xformers-0.0.25-cp310-cp310-manylinux2014_x86_64.whl

%cd /content
!git clone -b 23.05-devel https://github.com/NVIDIA/apex
%cd /content/apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

%cd /content
!git clone https://github.com/Dao-AILab/flash-attention
%cd /content/flash-attention
!pip install -v -e .

%cd /content
!git clone -b dev https://github.com/camenduru/Open-Sora
%cd /content/Open-Sora
!pip install -v -e .

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hpcai-tech/Open-Sora/resolve/main/OpenSora-v1-16x256x256.pth -d /content/Open-Sora/models -o OpenSora-v1-16x256x256.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hpcai-tech/Open-Sora/resolve/main/OpenSora-v1-HQ-16x256x256.pth -d /content/Open-Sora/models -o OpenSora-v1-HQ-16x256x256.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hpcai-tech/Open-Sora/resolve/main/OpenSora-v1-HQ-16x512x512.pth -d /content/Open-Sora/models -o OpenSora-v1-HQ-16x512x512.pth

!git clone https://huggingface.co/DeepFloyd/t5-v1_1-xxl /content/Open-Sora/pretrained_models/t5_ckpts/t5-v1_1-xxl

  Using cached https://download.pytorch.org/whl/cu121/torch-2.2.1%2Bcu121-cp310-cp310-linux_x86_64.whl (757.3 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl.metadata (1.8 kB)
  Using cached triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 753.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 6.5 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.5:
      Successfully uninstalled nvidia-nccl-cu12-2.20.5
  Attempting 

In [1]:
%cd /content/Open-Sora
# Sample 16x256x256 (5s/sample)
!torchrun --standalone --nproc_per_node 1 scripts/inference.py configs/opensora/inference/16x256x256.py --ckpt-path /content/Open-Sora/models/OpenSora-v1-16x256x256.pth

[Errno 2] No such file or directory: '/content/Open-Sora'
/content
y

/usr/bin/python3: can't open file '/content/scripts/inference.py': [Errno 2] No such file or directory
E1015 10:18:36.929000 139308237165184 torch/distributed/elastic/multiprocessing/api.py:833] failed (exitcode: 2) local_rank: 0 (pid: 3180) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/local/bin/torchrun", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 348, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 901, in main
    run(args)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 892, in run
    elastic_launch(
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/launcher/api.py", line 133, in __call__
    return launch_agent(self._config, self._entrypoint, list(ar

In [ ]:
%cd /content/Open-Sora
# Sample 16x512x512 (20s/sample, 100 time steps)
!torchrun --standalone --nproc_per_node 1 scripts/inference.py configs/opensora/inference/16x512x512.py --ckpt-path /content/Open-Sora/models/OpenSora-v1-HQ-16x512x512.pth

In [ ]:
%cd /content/Open-Sora
# Sample 64x512x512 (40s/sample, 100 time steps)
!torchrun --standalone --nproc_per_node 1 scripts/inference.py configs/opensora/inference/64x512x512.py --ckpt-path /content/Open-Sora/models/OpenSora-v1-HQ-16x512x512.pth

In [ ]:
%cd /content/Open-Sora
# Sample 64x512x512 with sequence parallelism (30s/sample, 100 time steps)
# sequence parallelism is enabled automatically when nproc_per_node is larger than 1
!torchrun --standalone --nproc_per_node 2 scripts/inference.py configs/opensora/inference/64x512x512.py --ckpt-path /content/Open-Sora/models/OpenSora-v1-HQ-16x512x512.pth